
Datasets:
- with LOAD_DATASETS=full, argilla loads `gutenberg_spacy-ner-monitoring` for Token Classification with default spaCy predictions ; which is a fork of https://huggingface.co/datasets/gutenberg_time
- default NER dataset in papers is CoNLL-2003 https://huggingface.co/datasets/conll2003
- https://huggingface.co/datasets/DFKI-SLT/few-nerd
- https://huggingface.co/datasets/tner/ontonotes5
- Look for argilla compatible NER datasets with this search: https://huggingface.co/datasets?task_categories=task_categories:token-classification&sort=trending&search=argilla


- ✨ Provide suggested spans with a confidence score, so your team doesn't need to start from scratch.


In [1]:
from typing import List, Tuple, Union, Dict
import types

In [2]:
import argilla as rg

rg.init(api_url="http://localhost:6900", api_key="admin.apikey")

/Users/louis.guitton/workspace/mlops-talk-llm-kg/venv/lib/python3.11/site-packages/argilla/client/client.py:178: UserWarning: No workspace configuration was detected. To work with Argilla datasets, specify a valid workspace name on `rg.init` or set it up through the `rg.set_workspace` function.
  warnings.warn(


## Push to Huggingface

In [155]:
dataset.push_to_huggingface(
    repo_id="louisguitton/dev-ner-ontonotes",split="validation"
)

Uploading the dataset shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.73it/s]
README.md: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10.0k/10.0k [00:00<00:00, 8.44MB/s]


## Add suggestions to a remote dataset

In [262]:
remote_dataset = rg.FeedbackDataset.from_argilla(
    name="dev-ner-ontonotes",
    workspace="admin",
    with_vectors="all"
)

In [263]:
from typing import Type
from argilla.client.feedback.dataset.remote.dataset import RemoteFeedbackDataset
from argilla.client.feedback.schemas.remote.records import RemoteFeedbackRecord, RemoteSuggestionSchema
from argilla.client.feedback.schemas.suggestions import SuggestionSchema

def labeller(nlp: Type[spacy.language.Language], text: str) -> List[SpanValueSchema]:
    """Generate NER preditions from a spaCy model in the Argilla format."""
    doc = nlp(text)
    return [
        SpanValueSchema(
            start=ent.start_char,
            end=ent.end_char,
            label=ent.label_,
            score=0
        )  for ent in doc.ents
    ]
    
def add_suggestions_to_remote_dataset(remote_dataset: RemoteFeedbackDataset, nlp: Type[spacy.language.Language]) -> None:
    """Add suggestions from a spaCy NER model to a remote instance of an existing Argilla dataset.
    
    ref: https://docs.argilla.io/en/latest/tutorials_and_integrations/tutorials/feedback/end2end_examples/add-suggestions-and-responses-005.html#For-the-RemoteFeedbackDataset"""
    modified_records: List[RemoteFeedbackRecord] = [record for record in remote_dataset.records]
    
    for record in modified_records:
        pred: List[SpanValueSchema] = labeller(nlp, record.fields["text"])
        # passing more than 1 suggestion fails with this error:
        # ValidationApiError: Argilla server returned an error with http status: 422. Error details: {'response': 'Record at 
        # position 0 is not valid because found duplicate suggestions question IDs', 'params': None}
        record.suggestions: Union[Tuple[Union[RemoteSuggestionSchema, SuggestionSchema]], List[Union[RemoteSuggestionSchema, SuggestionSchema]]] = [{
                "question_name": "entities",
                "value": pred,
                "agent": nlp.meta['name']
            }]
    
    remote_dataset.update_records(modified_records)

In [171]:
import spacy

nlp = spacy.load("en_core_web_sm")

/Users/louis.guitton/workspace/mlops-talk-llm-kg/venv/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/louis.guitton/workspace/mlops-talk-llm-kg/venv/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/louis.guitton/workspace/mlops-talk-llm-kg/venv/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [264]:
add_suggestions_to_remote_dataset(remote_dataset, nlp)

In [270]:
def test_one_suggestion_and_no_response():
    r = remote_dataset.records[2]
    pred: List[SpanValueSchema] = labeller(nlp, r.fields["text"])
    r.responses = []
    r.suggestions = [{
                "question_name": "entities",
                "value": pred,
                "agent": nlp.meta['name']
            }]
    remote_dataset.update_records([r])

In [271]:
test_one_suggestion_and_no_response()

## Compute metrics

In [289]:
from argilla.client.feedback.metrics.utils import get_responses_and_suggestions_per_user

In [291]:
remote_dataset = rg.FeedbackDataset.from_argilla(
    name="ner-lvl2",
    workspace="admin",
    with_vectors="all"
)

# responses_and_suggestions_per_user = get_responses_and_suggestions_per_user(dataset=remote_dataset, question_name="entities")

Extracting responses and suggestions per user: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 861253.39it/s]


In [298]:
hf_dataset = remote_dataset.format_as("datasets")

In [300]:
hf_dataset[0]

{'text': 'A Russian diver has found the bodies of three of the 118 sailors who were killed when the nuclear submarine Kursk sank in the Barents Sea .',
 'entities': [{'user_id': 'fe7c1b6a-5d30-41d5-bb56-6675cfbad12f',
   'value': {'start': [2, 40, 53, 108, 122],
    'end': [9, 45, 56, 113, 137],
    'label': ['NORP', 'CARDINAL', 'CARDINAL', 'PRODUCT', 'LOC'],
    'text': ['Russian', 'three', '118', 'Kursk', 'the Barents Sea']},
   'status': 'submitted'}],
 'entities-suggestion': {'start': [2, 40, 53, 108, 122],
  'end': [9, 45, 56, 113, 137],
  'label': ['NORP', 'CARDINAL', 'CARDINAL', 'PRODUCT', 'LOC'],
  'text': ['Russian', 'three', '118', 'Kursk', 'the Barents Sea'],
  'score': [None, None, None, None, None]},
 'entities-suggestion-metadata': {'type': None, 'score': None, 'agent': None},
 'external_id': None,
 'metadata': '{}'}

In [ ]:
from spacy.tokens import Doc
from spacy.training import Example

examples: List[Example] = []
for row in hf_dataset:
    text = row["text"]
    gold = row["entities"]
    pred = row["entities-suggestion"]

    # generate Doc with Doc.set_ents from a list of spans for the predicted suggestions
    # ref: https://spacy.io/api/doc#set_ents

    # generate Doc with Doc.set_ents from a list of spans for the gold responses
    
    example = Example(predicted, reference)
    examples.append(example)

In [296]:
from spacy.scorer import Scorer
scorer = Scorer()

In [ ]:
scores = scorer.score(examples)

## Football Articles
### Create (and Delete) empty dataset in Argilla for our task

In [212]:
remote_dataset = rg.FeedbackDataset.from_argilla("football-news", workspace="admin")
remote_dataset.delete()

In [213]:
dataset = rg.FeedbackDataset(
    fields=[
        rg.TextField(name="title"),
        rg.TextField(name="content"),
    ],
    questions=[
        rg.LabelQuestion(
            name="category",
            title="What is the category of the article?",
            labels=[
                "Coach Commentary", "Transfer News", "Match Report",
                "Player Profile", "League Updates", "Injury Updates",
                "Tactical Analysis", "Social Media Reaction",
                "Historical Milestone", "Match Incident"
            ],
            required=False,
            visible_labels=None
        ),
        rg.SpanQuestion(
            name="entities",
            title="Highlight the entities in the content:",
            labels=["Competition", "Team", "Player", "Match", "Transfer"],
            field="content",
            required=True,
            allow_overlapping=True
        )
    ],
    metadata_properties = [
        rg.TermsMetadataProperty(name="link"),
        rg.TermsMetadataProperty(name="source"),
    ],
    vectors_settings=[], # we will add sentence embeddings a posteriori
    guidelines="Please, read the question carefully and try to answer it as accurately as possible."
)

/Users/louis.guitton/workspace/mlops-talk-llm-kg/venv/lib/python3.11/site-packages/argilla/client/feedback/schemas/questions.py:416: UserWarning: `visible_labels=20` is greater than the total number of labels (5), so it will be set to `5`.
  warnings.warn(msg, UserWarning, stacklevel=1)


In [214]:
remote_dataset = dataset.push_to_argilla(name="football-news", workspace="admin")

[05/22/24 12:43:45] INFO     INFO:argilla.client.feedback.dataset.local.mixins:✓ Dataset succesfully  ]8;id=817101;file:///Users/louis.guitton/workspace/mlops-talk-llm-kg/venv/lib/python3.11/site-packages/argilla/client/feedback/dataset/local/mixins.py\mixins.py]8;;\:]8;id=649143;file:///Users/louis.guitton/workspace/mlops-talk-llm-kg/venv/lib/python3.11/site-packages/argilla/client/feedback/dataset/local/mixins.py#271\271]8;;\
                             pushed to Argilla                                                                     

                    INFO     INFO:argilla.client.feedback.dataset.local.mixins:RemoteFeedbackDataset( ]8;id=234094;file:///Users/louis.guitton/workspace/mlops-talk-llm-kg/venv/lib/python3.11/site-packages/argilla/client/feedback/dataset/local/mixins.py\mixins.py]8;;\:]8;id=591133;file:///Users/louis.guitton/workspace/mlops-talk-llm-kg/venv/lib/python3.11/site-packages/argilla/client/feedback/dataset/local/mixins.py#272\272]8;;\
                                id=764d5bd3-e075-4a5b-9e0e-1e3b69c4703c                                            
                                name=football-news                                                                 
                                workspace=Workspace(id=84a8fb6f-3350-4e9b-97c0-043cfedef934,                       
                             name=admin, inserted_at=2024-05-14 17:08:20.825501,                                   
                             updated_at=2024-05-14 17:08:20.825501)                                                
                                url=http://localhost:6900/dataset/764d5bd3-e075-4a5b-9e0e-1e3b69c4703              
                             c/annotation-mode                                                                     
                                fields=[RemoteTextField(id=UUID('8aba1023-dfbc-4208-805e-33583133e1a7              
                             '), client=None, name='title', title='Title', required=True,                          
                             type='text', use_markdown=False),                                                     
                             RemoteTextField(id=UUID('0b1fda52-583d-427d-9cb7-3060f12d0f2f'),                      
                             client=None, name='content', title='Content', required=True,                          
                             type='text', use_markdown=False)]                                                     
                                questions=[RemoteLabelQuestion(id=UUID('bafebe1e-f68a-4a71-a331-9e917              
                             41a3c4f'), client=None, name='category', title='What is the category of               
                             the article?', description=None, required=False, type='label_selection',              
                             labels=['Coach Commentary', 'Transfer News', 'Match Report', 'Player                  
                             Profile', 'League Updates', 'Injury Updates', 'Tactical Analysis',                    
                             'Social Media Reaction', 'Historical Milestone', 'Match Incident'],                   
                             visible_labels=None),                                                                 
                             RemoteSpanQuestion(id=UUID('9c0abcb0-48b1-494e-b308-c75a8ef256e5'),                   
                             client=None, name='entities', title='Highlight the entities in the                    
                             content:', description=None, required=True, type='span',                              
                             field='content', labels=[SpanLabelOption(value='Competition',                         
                             text='Competition', description=None), SpanLabelOption(value='Team',                  
                             text='Team', description=None), SpanLabelOption(value='Player',                       
                             text='Player', description=None), SpanLabelOption(value='Match',                      
                             text='Match', description=None), SpanLabelOption(value='Transfer',                    
                             text='Transfer', description=None)], visible_labels=5,                                
                             allow_overlapping=True)]                                                              
                                guidelines=Please, read the question carefully a

### Add bare records to the remote Argilla dataset

In [215]:
from typing import Iterator
from tqdm import tqdm

def records_generator(filepath: str = '../../data/football-news-articles/final-articles.csv') -> Iterator[rg.FeedbackRecord]:
    """Create Argilla records from the Football News kaggle dataset.
    
    Notes:
    - articles with source="all-football-app" have encoding issues
    """
    dataset: pd.DataFrame = (
        pd.read_csv(filepath)
        .loc[lambda d: d.source.isin(["skysports", "the-analyst"])]
    )

    for index, row in tqdm(dataset.iterrows()):
        record = rg.FeedbackRecord(
            fields={
                "title": row['title'],
                "content": row['content']
            },
            metadata={
                "link": row['link'],
                "source": row['source'],
            },
            vectors={},
            responses=[],
            suggestions=[],
            external_id=index,
        )

        yield record

In [216]:
LIMIT = 50

In [217]:
remote_dataset = rg.FeedbackDataset.from_argilla("football-news", workspace="admin")
remote_dataset.add_records(list(records_generator())[:LIMIT])

2157it [00:00, 33218.70it/s]

/Users/louis.guitton/workspace/mlops-talk-llm-kg/venv/lib/python3.11/site-packages/rich/live.py:231: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

### Add vectors to records to enable Similarity Search in Argilla

In [218]:
from argilla.client.feedback.integrations.sentencetransformers import SentenceTransformersExtractor

In [219]:
FAST_AND_SMALL = "sentence-transformers/all-MiniLM-L6-v2"

ste = SentenceTransformersExtractor(
    model=FAST_AND_SMALL,
    show_progress=True,
)

/Users/louis.guitton/workspace/mlops-talk-llm-kg/venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [220]:
remote_dataset = rg.FeedbackDataset.from_argilla("football-news", workspace="admin")

# Update the dataset
remote_dataset = ste.update_dataset(
    dataset=remote_dataset,
    fields=None, # None means using all fields
    update_records=True, # Also, update the records in the dataset
    overwrite=True, # Whether to overwrite existing vectors
)

Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  4.35it/s]


## Add suggestions for the text classification question using a zero-shot classification model from HuggingFace
If we had labels, we could train a model that uses sentence embeddings (for more details, see [this argilla tutorial](https://docs.argilla.io/en/v1.28.0/tutorials_and_integrations/tutorials/feedback/labelling-feedback-setfit.html))

```python
from setfit import SetFitModel, SetFitTrainer

model = SetFitModel.from_pretrained("all-MiniLM-L6-v2")
trainer = SetFitTrainer(model=model, train_dataset=train_dataset)
```

But for the cold start, we look for a small model on [HuggingFace](https://huggingface.co/models?pipeline_tag=zero-shot-classification)

In [221]:
from transformers import pipeline

# model_name = "sileod/deberta-v3-small-tasksource-nli"
# model_name = "cointegrated/rubert-tiny-bilingual-nli"
# model_name = "typeform/distilbert-base-uncased-mnli"
model_name = "valhalla/distilbart-mnli-12-3"
classifier = pipeline("zero-shot-classification", model=model_name)

In [222]:
text = "one day I will see the world"
candidate_labels = ['travel', 'cooking', 'dancing']
classifier(text, candidate_labels)

{'sequence': 'one day I will see the world',
 'labels': ['travel', 'dancing', 'cooking'],
 'scores': [0.985403299331665, 0.007384597323834896, 0.007212089374661446]}

In [ ]:
remote_dataset = rg.FeedbackDataset.from_argilla("football-news", workspace="admin")

modified_records = [record for record in remote_dataset.records]
for record in tqdm(modified_records):
    x = ". ".join([record.fields["title"], record.fields["content"]])
    # if x: str, then y = {"sequence": 'one day I will see the world', "labels": ['travel', 'dancing', 'cooking'], 'scores': [0.8434, 0.0814, 0.0750]}
    y = classifier(
        sequences=x,
        candidate_labels=remote_dataset.question_by_name("category").labels
    )    
    record.suggestions = [
        {
            "question_name": "category",
            "value": y["labels"][0],
            "agent": model_name,
            "score": y["scores"][0]
        }
    ]

remote_dataset.update_records(modified_records)

  8%|████████████▏                                                                                                                                            | 4/50 [00:23<04:24,  5.74s/it]

Problems:
- accuracy is not great in practice
- inference time is high

Approach: 
- label a few (10s) articles and train a classifier with SetFit 

## Add suggestions for the token classification question using a zero-shot NER model from HuggingFace

In [ ]:
remote_dataset = rg.FeedbackDataset.from_argilla("football-news", workspace="admin")

In [ ]:
import spacy
from gliner_spacy.pipeline import (  # noqa: F401 because we need to register the factory with spacy
    GlinerSpacy,
)

candidate_labels =[label.value.lower() for label in remote_dataset.question_by_name("entities").labels] # NuZero requires labels to be lower-cased

model_name = "numind/NuZero_token"

nlp = spacy.blank("en")
nlp.add_pipe("gliner_spacy", config={
    "gliner_model": model_name,
    "chunk_size": 250,
    "labels": candidate_labels,  
    "style": "ent",
    "threshold": 0.3,
})

In [ ]:
from argilla.client.feedback.schemas import SpanValueSchema

modified_records = [record for record in remote_dataset.records]
for record in tqdm(modified_records):
    doc = nlp(record.fields["content"]) 
    record.suggestions = [
        {
            "question_name": "entities",
            "value": [
                SpanValueSchema(
                    start=ent.start_char,
                    end=ent.end_char,
                    label=ent.label_.capitalize(),
                )
                for ent in doc.ents
            ],
            "agent": model_name,
            # "score": ...
        }
    ] + [dict(s) for s in record.suggestions]  # we keep existing suggestions

remote_dataset.update_records(modified_records)